In [13]:
import pprint as pp
import json
import json
import os
import pprint
import plotly.express as px
from itertools import chain
import pandas as pd
import requests
import pprint
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [14]:
# starting code that loads data, run this cell, but no need to understand it in depth
# this function just loads the data from files
def load_json_file_named(file_name):
    try: 
        loaded_data = []
        file_location = f"{file_name}"
        with open(file_location, 'r') as file: # or f"data/{file_name}" depending on your files
            loaded_data =  json.load(file)
    except OSError as e:
        print(f"Error. Does the file exist in this folder? {file_location}\n\n {e}")
    return loaded_data

In [15]:
def plot_time_series(input_dictionary,key):
    number_of_new_positive=[]
    number_of_new_deaths=[]
    number_of_first_infections=[]
    number_of_reinfections=[]
    dates=[]
    for part_dictionary in input_dictionary:
        if part_dictionary['locations'][key]=={}:
            number_of_new_positive.append(0)
            number_of_new_deaths.append(0)
            number_of_first_infections.append(0)
            number_of_reinfections.append(0)
            dates.append(datetime.strptime(str(part_dictionary['date']),"%Y%m%d"))
        else:
            number_of_new_positive.append(part_dictionary['locations'][key]['new_positive'])
            number_of_new_deaths.append(part_dictionary['locations'][key]['new_deaths'])
            number_of_first_infections.append(part_dictionary['locations'][key]['first_infections'])
            number_of_reinfections.append(part_dictionary['locations'][key]['reinfections'])
            dates.append(datetime.strptime(str(part_dictionary['date']),"%Y%m%d"))
    fig = make_subplots(rows=4, cols=1)
    fig.add_trace(go.Scatter(x=dates, y=number_of_new_positive,name="Number of New Positive Cases"),
    row=1, col=1)
    fig.add_trace(go.Scatter(x=dates, y=number_of_new_deaths,name=" Number Of New Deaths"),
    row=2, col=1)
    fig.add_trace(go.Scatter(x=dates, y=number_of_first_infections,name="Number of First Infections Cases"),
    row=3, col=1)
    fig.add_trace(go.Scatter(x=dates, y=number_of_reinfections,name="Number of Reinfections"),
    row=4, col=1)
    fig.update_layout(height=800,width=1000,title_text=str(key)+" : Time series graphs")
    fig.show()

In [16]:
def extract_all_values(dict_temp, dimension):
    value_list_def = list(value for key, value in dict_temp.items())
    key_list_def = list(key for key, value in dict_temp.items())
    result = key_list_def if dimension == "key" else value_list_def
    return result

In [17]:
boards_info = load_json_file_named('nhs_scotland_boards.json')
covid_days = load_json_file_named('covid_records_scotland.json')
cancer_waiting_times = load_json_file_named('cancer_waiting_times.json')
boards_code_lookup = load_json_file_named('boards_code_lookup.json')

In [7]:
budget_count = list(chain(*list(extract_all_values(i['budget_millions'], "values") for i in boards_info)))
regions = list(i['name'] for i in boards_info)
year_mapping = list(chain(*list(extract_all_values(i['budget_millions'], "key") for i in boards_info)))
staff_count = list(extract_all_values(i['people'], "values") for i in boards_info)
staff_count = [list(i) for i in zip(*staff_count)][1]
area_in_km2 = list(extract_all_values(i['geographic'], "values") for i in boards_info)
area_in_km2 = [list(i) for i in zip(*area_in_km2)][0]
staff_per_area = [round(staff/area,2) for staff,area in zip(staff_count, area_in_km2)]
data_sunburst = dict(
    budget_count = budget_count,
    regions = list(i for i in regions for _ in range(3)) ,
    years = year_mapping
)
fig = px.sunburst(data_sunburst,path= ['regions', 'years'], values = 'budget_count',color = 'budget_count', title = "SunBurst Chart of Accumulated Budgets over years for different NHS bases")
fig.show()

In [18]:
for key in covid_days[0]['locations'].keys():
    if key=='date':
        continue
    else:
        plot_time_series(covid_days,key)

In [12]:
covid_days[0]

{'date': 20200228,
 'whole_scotland': {'new_positive': 1,
  'new_deaths': 0,
  'first_infections': 1,
  'reinfections': 0},
 'locations': {'NHS Ayrshire and Arran': {},
  'NHS Borders': {},
  'NHS Dumfries and Galloway': {},
  'NHS Fife': {},
  'NHS Forth Valley': {},
  'NHS Grampian': {},
  'NHS Greater Glasgow and Clyde': {},
  'NHS Highland': {},
  'NHS Lanarkshire': {},
  'NHS Lothian': {},
  'NHS Orkney': {},
  'NHS Shetland': {},
  'NHS Tayside': {'new_positive': 1,
   'new_deaths': 0,
   'first_infections': 1,
   'reinfections': 0},
  'NHS Western Isles': {}}}